Preprocessing:
We load weekly returns for the selected assets, align the data, and compute the annualized statistics that will feed the optimization problem.

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.optimize import minimize
from IPython.display import display

pd.set_option('display.float_format', '{:.4f}'.format)

FREQ = 52
TARGET_MEAN_ANNUAL = 0.20
STOCK_TICKERS = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AVGO', 'BRK/B', 'LLY']
ETF_TICKER = 'SPY'
ALL_ASSETS = STOCK_TICKERS + [ETF_TICKER]

DATA_PATH = '/content/spx_returns_weekly.xlsx'

print(f'Using data file: {DATA_PATH}')

Using data file: /content/spx_returns_weekly.xlsx


In [4]:
stocks = pd.read_excel(DATA_PATH, sheet_name='s&p500 rets')
benchmarks = pd.read_excel(DATA_PATH, sheet_name='benchmark rets')

returns = (
    stocks[['date'] + STOCK_TICKERS]
    .merge(benchmarks[['date', ETF_TICKER]], on='date')
    .dropna()
    .set_index('date')
    .sort_index()
)

weekly_mean = returns.mean()
weekly_cov = returns.cov()

annual_mean = weekly_mean * FREQ
annual_cov = weekly_cov * FREQ
annual_vol = returns.std() * np.sqrt(FREQ)

provided_meta = pd.DataFrame([
    ('AAPL', 'Apple Inc', 3.008822e12),
    ('NVDA', 'NVIDIA Corp', 3.480172e12),
    ('MSFT', 'Microsoft Corp', 3.513735e12),
    ('GOOGL', 'Alphabet Inc', 2.145918e12),
    ('AMZN', 'Amazon.com Inc', 2.303536e12),
    ('META', 'Meta Platforms Inc', 1.745094e12),
    ('TSLA', 'Tesla Inc', 9.939227e11),
    ('AVGO', 'Broadcom Inc', 1.148592e12),
    ('BRK/B', 'Berkshire Hathaway Inc', 1.064240e12),
    ('LLY', 'Eli Lilly & Co', 7.332726e11),
    ('SPY', 'SPDR S&P 500 ETF Trust', np.nan),
], columns=['ticker', 'name', 'mkt cap']).set_index('ticker')

asset_stats = (
    pd.DataFrame({
        'annual_mean': annual_mean,
        'annual_vol': annual_vol,
        'sharpe': annual_mean / annual_vol,
    })
    .join(provided_meta, how='left')
    .loc[ALL_ASSETS, ['name', 'mkt cap', 'annual_mean', 'annual_vol', 'sharpe']]
)

returns.head()

,AAPL,NVDA,MSFT,GOOGL,AMZN,META,TSLA,AVGO,BRK/B,LLY,SPY
date,,,,,,,,,,,
2015-01-09,0.0245,-0.0093,0.0092,-0.0544,-0.0375,-0.0091,-0.0577,0.0480,0.0020,-0.0019,-0.0057
2015-01-16,-0.0537,0.0008,-0.0201,0.0194,-0.0209,-0.0329,-0.0658,-0.0103,-0.0017,0.0107,-0.0128
2015-01-23,0.0660,0.0376,0.0203,0.0617,0.0744,0.0353,0.0426,0.0305,-0.0006,0.0205,0.0166
2015-01-30,0.0370,-0.0726,-0.1437,-0.0081,0.1349,-0.0247,0.0115,-0.0383,-0.0349,-0.0018,-0.0269
2015-02-06,0.0191,0.0623,0.0498,-0.0068,0.0557,-0.0190,0.0676,0.0180,0.0436,-0.0228,0.0306


In [5]:
asset_stats

,name,mkt cap,annual_mean,annual_vol,sharpe
AAPL,Apple Inc,3008822000000.0000,0.2387,0.2766,0.8629
NVDA,NVIDIA Corp,3480172000000.0000,0.6456,0.4633,1.3935
MSFT,Microsoft Corp,3513735000000.0000,0.2614,0.2400,1.0893
GOOGL,Alphabet Inc,2145918000000.0000,0.2168,0.2799,0.7747
AMZN,Amazon.com Inc,2303536000000.0000,0.2934,0.3060,0.9590
META,Meta Platforms Inc,1745094000000.0000,0.2619,0.3513,0.7455
TSLA,Tesla Inc,993922700000.0000,0.4698,0.5864,0.8010
AVGO,Broadcom Inc,1148592000000.0000,0.3949,0.3751,1.0526
BRK/B,Berkshire Hathaway Inc,1064240000000.0000,0.1350,0.1907,0.7082
LLY,Eli Lilly & Co,733272600000.0000,0.2815,0.2830,0.9949


## 1.1 Constrained mean-variance portfolio

We solve for the minimum-variance portfolio that achieves the 20% annual target return while respecting the weight bounds (no position below -20% or above 35%) and the budget constraint. The Sharpe ratio assumes a zero risk-free rate.

In [6]:
def feasible_start(mu, target):
    mu = np.asarray(mu, dtype=float)
    A = np.vstack([np.ones(mu.shape[0]), mu])
    b = np.array([1.0, target], dtype=float)
    return A.T @ np.linalg.solve(A @ A.T, b)

def solve_min_var(mu, cov, target, bounds=None):
    mu = np.asarray(mu, dtype=float)
    cov = np.asarray(cov, dtype=float)
    x0 = feasible_start(mu, target)
    constraints = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},
        {'type': 'eq', 'fun': lambda w: mu.dot(w) - target},
    )
    result = minimize(lambda w: w @ cov @ w, x0, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_stats(weights, mu, cov):
    weights = np.asarray(weights, dtype=float)
    mu = np.asarray(mu, dtype=float)
    cov = np.asarray(cov, dtype=float)
    mean = float(mu.dot(weights))
    variance = float(weights @ cov @ weights)
    vol = variance ** 0.5
    sharpe = mean / vol if vol > 0 else np.nan
    return {'mean (annual)': mean, 'volatility (annual)': vol, 'Sharpe ratio': sharpe}

In [7]:
bounds = [(-0.20, 0.35)] * len(ALL_ASSETS)

constrained_solution = solve_min_var(annual_mean.values, annual_cov.values, TARGET_MEAN_ANNUAL, bounds=bounds)
constrained_weights = pd.Series(constrained_solution.x, index=ALL_ASSETS, name='weight')
constrained_stats = portfolio_stats(constrained_weights.values, annual_mean.values, annual_cov.values)

display(constrained_weights.sort_values(ascending=False).to_frame())
display(pd.Series(constrained_stats, name='Constrained portfolio'))

,weight
BRK/B,0.3500
LLY,0.2133
SPY,0.1513
MSFT,0.1447
AMZN,0.0938
AVGO,0.0355
AAPL,0.0292
GOOGL,0.0088
META,0.0024
NVDA,-0.0138


,Constrained portfolio
mean (annual),0.2000
volatility (annual),0.1649
Sharpe ratio,1.2129


## 1.2 Weight extremes vs asset Sharpe ratios

We compare the constrained allocations with each asset's standalone annualized mean and Sharpe ratio.

In [8]:
weight_vs_sharpe = (
    pd.concat([constrained_weights, asset_stats[['annual_mean', 'sharpe']]], axis=1)
    .rename(columns={'annual_mean': 'asset annual mean', 'sharpe': 'asset Sharpe'})
    .assign(abs_weight=lambda df: df['weight'].abs())
    .sort_values('abs_weight', ascending=False)
)

display(weight_vs_sharpe[['weight', 'abs_weight', 'asset annual mean', 'asset Sharpe']])

,weight,abs_weight,asset annual mean,asset Sharpe
BRK/B,0.3500,0.3500,0.1350,0.7082
LLY,0.2133,0.2133,0.2815,0.9949
SPY,0.1513,0.1513,0.1313,0.7682
MSFT,0.1447,0.1447,0.2614,1.0893
AMZN,0.0938,0.0938,0.2934,0.9590
AVGO,0.0355,0.0355,0.3949,1.0526
AAPL,0.0292,0.0292,0.2387,0.8629
TSLA,-0.0153,0.0153,0.4698,0.8010
NVDA,-0.0138,0.0138,0.6456,1.3935
GOOGL,0.0088,0.0088,0.2168,0.7747


The largest position hits the 35% upper bound on BRK/B even though its standalone Sharpe ratio (~0.71) is one of the lowest in the set; its low volatility and comparatively weak correlation with the high-growth names help the optimizer reach the variance minimum while still meeting the mean target. Eli Lilly and SPY also carry substantial long weights thanks to strong risk-adjusted returns and diversification benefits.

On the other hand, NVDA sports the highest standalone Sharpe (~1.39) yet the optimizer keeps only a small short (about -1.4%). With several growth-oriented names already in the mix, adding more NVDA would push the portfolio's mean above target and inflate variance because of high correlations. The optimizer instead offsets that exposure with slight shorts in NVDA and TSLA to fine-tune the target-return constraint while keeping volatility low.

## 1.3 Effect of removing the inequality bounds

We now solve the same problem without the -20% / +35% bounds and compare the allocations and portfolio statistics.

In [9]:
unbounded_solution = solve_min_var(annual_mean.values, annual_cov.values, TARGET_MEAN_ANNUAL, bounds=None)
unbounded_weights = pd.Series(unbounded_solution.x, index=ALL_ASSETS, name='unbounded weight')
unbounded_stats = portfolio_stats(unbounded_weights.values, annual_mean.values, annual_cov.values)

weights_comparison = pd.concat(
    [constrained_weights.rename('bounded weight'), unbounded_weights], axis=1
)
weights_comparison['difference'] = weights_comparison['bounded weight'] - weights_comparison['unbounded weight']

stats_comparison = pd.DataFrame({'bounded': constrained_stats, 'unbounded': unbounded_stats})

display(weights_comparison)
display(stats_comparison)

,bounded weight,unbounded weight,difference
AAPL,0.0292,0.0289,0.0003
NVDA,-0.0138,-0.0149,0.0011
MSFT,0.1447,0.1523,-0.0077
GOOGL,0.0088,0.0075,0.0014
AMZN,0.0938,0.0950,-0.0012
META,0.0024,0.0019,0.0005
TSLA,-0.0153,-0.0152,-0.0001
AVGO,0.0355,0.0348,0.0007
BRK/B,0.3500,0.3686,-0.0186
LLY,0.2133,0.2112,0.0021


,bounded,unbounded
mean (annual),0.2000,0.2000
volatility (annual),0.1649,0.1649
Sharpe ratio,1.2129,1.2131


Without the bounds the optimizer places roughly 36.9% in BRK/B, so the 35% ceiling is the only active constraint. To preserve the target mean after capping BRK/B at 35%, the bounded solution shifts a few percentage points toward MSFT, SPY, and LLY and trims NVDA slightly. The portfolio volatility is the same (0.1649 vs. 0.1649 annualized) and the Sharpe ratio slips by less than 0.0002, confirming that the inequality constraint has a minimal cost in this case.

Answering the questions in the write-up,

- BRK/B provides the main diversification benefit and is pinned at the upper bound in the constrained solution.
- Small short positions in NVDA and TSLA help satisfy the mean target while containing volatility.
- Relaxing the bounds barely improves efficiency, so the practical impact of the constraints is minor for this asset set.